In [ ]:
import requests # type: ignore
import base64
from bs4 import BeautifulSoup # type: ignore
from configparser import ConfigParser 
import base64

In [ ]:
# Read configuration

config = ConfigParser()
config.read('config.ini')
PASSWORD = config['DEFAULT']['PASSWORD']
USERNAME = config['DEFAULT']['USERNAME']
URL = config['DEFAULT']['URL']


In [ ]:
def get_catalog_contents(response):
    print(f"response: {response}")
    b_xml = BeautifulSoup(response, 'html.parser')
    catalog = []
    for path in b_xml.find_all('absolutepath'):
        for data in path.children:
            catalog.append(data)
    return catalog

def list_catalog_contents(path='/Custom/Interfaces'):
    payload = f'''<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:v2="http://xmlns.oracle.com/oxp/service/v2">
       <soapenv:Header/>
       <soapenv:Body>
          <v2:getFolderContents>
             <v2:folderAbsolutePath>{path}</v2:folderAbsolutePath>
             <v2:userID>{USERNAME}</v2:userID>
             <v2:password>{PASSWORD}</v2:password>
          </v2:getFolderContents>
       </soapenv:Body>
    </soapenv:Envelope>
    </soapenv:Envelope>
    '''
    
    header = {
        'Content-type':'text/xml; charset=utf-8'
    }
    
    response = requests.request("POST", url=URL, data=payload)
    
    if response.status_code == 200:
        return get_catalog_contents(response.content)
    else:
        return response


In [ ]:
def save(content, obj_name):
    decoded =  base64.b64decode(content)
    with open(f"./Inputs/{obj_name}", 'bw+') as file:
        file.write(bytes(decoded))
        
def get_base64_objects(response, object_name):
    xml = BeautifulSoup(response, 'html.parser')
    content = xml.find('downloadobjectreturn')
    objs = []
    for _content in content.contents:
        objs.append(_content)
        save(_content, object_name)
    return 

def download_object(object_name):
    payload = f'''<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:v2="http://xmlns.oracle.com/oxp/service/v2">
       <soapenv:Header/>
       <soapenv:Body>
          <v2:downloadObject>
             <v2:reportAbsolutePath>{object_name}</v2:reportAbsolutePath>
             <v2:userID>{USERNAME}</v2:userID>
             <v2:password>{PASSWORD}</v2:password>
          </v2:downloadObject>
       </soapenv:Body>
    </soapenv:Envelope>
    '''
    header = {
        'Content-type':'text/xml; charset=utf-8'
    }
    
    response = requests.request("POST", url=URL, headers=header, data=payload)

    if response.status_code:
        print(response.content)
        return get_base64_objects(response.content, object_name[object_name.rfind('/') + 1:])


In [ ]:
list_of_folders = [
    "/Custom"
]

In [ ]:
for folder in list_of_folders:
    print(f"folder: {folder}")
    for f_catalog in list_catalog_contents(folder):
        if f_catalog not in list_of_folders:
            print(f"Downloading now: {f_catalog}")
            download_object(f_catalog)